In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('output/kaggle/working//model1'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import re
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Use spacy for one go
import spacy
nlp = spacy.load('en_core_web_lg')

np.random.seed(42)

In [ ]:
train = pd.read_csv('/kaggle/input/commonlitreadabilityprize/train.csv')
test = pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv')
submission = pd.read_csv('/kaggle/input/commonlitreadabilityprize/sample_submission.csv')

In [ ]:
# Function to up url_legal category
def clean_url(data):
    if type(data) == float:
        return "None"
    else:
        data = re.sub(r"https://|/wiki/|/|.org|pdf|wp-cont|-|%20|%|www.|frym.", " ", data)
        data = re.sub("[^a-zA-Z -]", " ", data.lower())
        data = re.sub("\s+", " ", data)
    return data

# Function to clean up text category
def clean_text(data):
    if type(data) == float:
        pass
    else:
        data = re.sub(r'\n|\\', " ", data.lower())
        data = re.sub("[^a-zA-Z]", " ", data.lower())
        data = re.sub("\s+", " ", data)
    return data

# Function to get sentences from input text
def get_sentences(data):
    row_doc = nlp(data)
    sents = list(row_doc.sents)
    return sents
# This will probably nor be helpful in this case, more useful for sentiment than complexity

# Function to generate tokens from input text
def gen_tokens(data):
    row_doc = nlp(data)
    return row_doc

# Removes stop words from tokenized text
def remove_stopwords(data):
    n_list = []
    for i in data:
        n_list.append([j.vector for j in i if not j.is_stop])
    return n_list

def remove_stopwords_url(data):
    n_list = []
    for i in data:
        n_list.append([j.text for j in i if not j.is_stop])
    return n_list

In [ ]:
# work = 'Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, “and what is the use of a book,” thought Alice “without pictures or conversations?” So she was considering in her own mind (as well as she could, for the hot day made her feel very sleepy and stupid), whether the pleasure of making a daisy-chain would be worth the trouble of getting up and picking the daisies, when suddenly a White Rabbit with pink eyes ran close by her. There was nothing so very remarkable in that; nor did Alice think it so very much out of the way to hear the Rabbit say to itself, “Oh dear! Oh dear! I shall be late!” (when she thought it over afterwards, it occurred to her that she ought to have wondered at this, but at the time it all seemed quite natural); but when the Rabbit actually took a watch out of its waistcoat-pocket, and looked at it, and then hurried on, Alice started to her feet, for it flashed across her mind that she had never before seen a rabbit with either a waistcoat-pocket, or a watch to take out of it, and burning with curiosity, she ran across the field after it, and fortunately was just in time to see it pop down a large rabbit-hole under the hedge. In another moment down went Alice after it, never once considering how in the world she was to get out again. The rabbit-hole went straight on like a tunnel for some way, and then dipped suddenly down, so suddenly that Alice had not a moment to think about stopping herself before she found herself falling down a very deep well. Either the well was very deep, or she fell very slowly, for she had plenty of time as she went down to look about her and to wonder what was going to happen next. First, she tried to look down and make out what she was coming to, but it was too dark to see anything; then she looked at the sides of the well, and noticed that they were filled with cupboards and book-shelves; here and there she saw maps and pictures hung upon pegs. She took down a jar from one of the shelves as she passed; it was labelled “ORANGE MARMALADE”, but to her great disappointment it was empty: she did not like to drop the jar for fear of killing somebody underneath, so managed to put it into one of the cupboards as she fell past it. “Well!” thought Alice to herself, “after such a fall as this, I shall think nothing of tumbling down stairs! How brave they’ll all think me at home! Why, I wouldn’t say anything about it, even if I fell off the top of the house!” (Which was very likely true.) Down, down, down. Would the fall never come to an end? “I wonder how many miles I’ve fallen by this time?” she said aloud. “I must be getting somewhere near the centre of the earth. Let me see: that would be four thousand miles down, I think—” (for, you see, Alice had learnt several things of this sort in her lessons in the schoolroom, and though this was not a very good opportunity for showing off her knowledge, as there was no one to listen to her, still it was good practice to say it over) “—yes, that’s about the right distance—but then I wonder what Latitude or Longitude I’ve got to?” (Alice had no idea what Latitude was, or Longitude either, but thought they were nice grand words to say.)'

In [ ]:
# Split train into X and Y
X = train.drop(columns='target')
Y = train['target']
# Apply clean_url function to train & test 'url_legal'
# clean_urls_train = [clean_url(i) for i in X['url_legal']]
# clean_urls_test = [clean_url(i) for i in test['url_legal']]
# Apply clean_text to train & test 'excerpt'
clean_text_train = [clean_text(i) for i in X['excerpt']]
clean_text_test = [clean_text(i) for i in test['excerpt']]

#clean_work = clean_text(work)

# Add cleaned columns back
# X['clean_url'] = clean_urls_train
X['clean_text'] = clean_text_train
# test['clean_url'] = clean_urls_test
test['clean_text'] = clean_text_test
# Generate tokens from clean_text
# train_url_tokens = remove_stopwords_url([gen_tokens(i) for i in X['clean_url']])
train_excerpt_tokens = remove_stopwords([gen_tokens(i) for i in X['clean_text']])
# test_url_tokens = remove_stopwords_url([gen_tokens(i) for i in test['clean_url']])
test_excerpt_tokens = remove_stopwords([gen_tokens(i) for i in test['clean_text']])

#clean_work_tokens = remove_stopwords(gen_tokens(clean_work))

# Add tokenized columns to train & test
# X['url_tokens'] = train_url_tokens
X['excerpt_tokens'] = train_excerpt_tokens
# test['url_tokens'] = test_url_tokens
test['excerpt_tokens'] = test_excerpt_tokens
# Drop uneeded
# X.drop(columns=['url_legal', 'excerpt', 'clean_text', 'id', 'standard_error'], inplace=True)
# test.drop(columns=['url_legal', 'excerpt', 'clean_text', 'id'], inplace=True)
# Split train into train & validation sets
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.1, random_state=42)
print("Train:",(X_train.shape, y_train.shape), "Val:",(X_val.shape, y_val.shape))

In [ ]:
X_train_vect = [np.array(i) for i in X_train['excerpt_tokens']]
X_val_vect = [np.array(i) for i in X_val['excerpt_tokens']]
test_vect = [np.array(i) for i in test['excerpt_tokens']]

#clean_vect = np.array(clean_work_tokens)

In [ ]:
def pad_array(data):
    data_out = []
    for i in range(len(data)):
        an_array = data[i]
        shape = np.shape(an_array)
        padded_array = np.zeros((200,300))
        padded_array[:shape[0],:shape[1]] = an_array
        data_out.append(padded_array)
    return data_out

In [ ]:
cool_array_X = pad_array(X_train_vect)
cool_array_V = pad_array(X_val_vect)
cool_array_test = pad_array(test_vect)
t = np.array(cool_array_X).reshape(len(X_train), 200, 300)
v = np.array(cool_array_V).reshape(len(y_val), 200, 300)
vt = np.array(cool_array_test).reshape(len(test), 200, 300)

#cw_t = pad_array(clean_vect)
#cw = np.array(cw_t).reshape(len())

y = np.array(y_train)
y = y.reshape(1, -1)

In [ ]:
model1 = Sequential()

model1.add(Bidirectional(LSTM(512, input_shape=(None, 300), return_sequences=True),
                            backward_layer=LSTM(512, go_backwards=True, return_sequences=True)))

model1.add(Dense(128, activation='relu'))

model1.add(Dense(1))

model1.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])



In [ ]:
model1.fit(t, y_train,
           validation_data=(v, y_val),
           batch_size = 20,
           epochs=100,
           verbose=1,
           
          )

In [ ]:
model1.save('model1')

In [ ]:
l = model1.predict(vt)

In [ ]:
f = [np.mean(i) for i in l]

In [ ]:
f

In [ ]:
submission['target'] = f

submission.to_csv("./submission.csv", index=False)